<a href="https://colab.research.google.com/github/Shaurov05/DeepLearningMaliciousURLs/blob/master/website_CNN3_success1(06).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # daimport pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
%matplotlib inline 
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils
import re
from keras.preprocessing import sequence
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence
from sklearn.svm import LinearSVC


Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
names=['URL','Category']
#df=pd.read_csv( "../input/website-classification-using-url/URL Classification.csv")
#df=pd.read_csv('../input/Website classification using URL/URL Classification.csv')
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/thesis/URL Classification.csv',names=names, na_filter=False)

In [5]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
lb.fit(df['Category'])
df['Category'] = lb.transform(df['Category'])

data = pd.get_dummies(df,prefix=['Category'], columns = ['Category'])
df = data
df[:2]

,URL,Category_0,Category_1,Category_2,Category_3,Category_4,Category_5,Category_6,Category_7,Category_8,Category_9,Category_10,Category_11,Category_12,Category_13,Category_14
1,http://www.liquidgeneration.com/,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,http://www.onlineanime.org/,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
df1 = df[1:4000]
df2 = df[50000:54000]
df3 = df[520000:524000]
df4 =df[535300:539300]
df5 = df[650000:654000]
df6= df[710000:714000]
df7=  df[764200:768200]
df8=  df[793080:797080]
df9=  df[839730:843730]
df10=  df[850000:854000]
df11=  df[955250:959250]
df12=  df[1013000:1017000]
df13=  df[1143000:1147000]
df14=  df[1293000:1297000]
df15=  df[1492000:1496000]
#df6 = df[77000:1562978]
dt=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15], axis=0)
df.drop(df.index[1:4000],inplace= True)
df.drop(df.index[50000:54000],inplace= True)
df.drop(df.index[520000:524000],inplace= True)
df.drop(df.index[535300:539300],inplace= True)
df.drop(df.index[650000:654000],inplace= True)
df.drop(df.index[710000:714000],inplace= True)
df.drop(df.index[764200:768200],inplace= True)
df.drop(df.index[793080:797080],inplace= True)
df.drop(df.index[839730:843730],inplace= True)
df.drop(df.index[850000:854000],inplace= True)
df.drop(df.index[955250:959250],inplace= True)
df.drop(df.index[1013000:1017000],inplace= True)
df.drop(df.index[1143000:1147000],inplace= True)
df.drop(df.index[1293000:1297000],inplace= True)
df.drop(df.index[1492000:1496000],inplace= True)
df.tail()

,URL,Category_0,Category_1,Category_2,Category_3,Category_4,Category_5,Category_6,Category_7,Category_8,Category_9,Category_10,Category_11,Category_12,Category_13,Category_14
1562974,http://www.maxpreps.com/,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1562975,http://www.myscore.com/,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1562976,http://sportsillustrated.cnn.com/highschool,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1562977,http://rss.cnn.com/rss/si_highschool?format=xml,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1562978,http://www.usatoday.com/sports/preps/,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [7]:
X_train=df['URL']
y_train=df.iloc[: , 1:16].values
print(y_train)
y_train.shape

[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]


(1502979, 15)

In [8]:
X_test=dt['URL']
y_test=dt.iloc[: , 1:16].values
print(y_test)
y_test.shape

[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]


(59999, 15)

In [9]:
from keras.preprocessing.text import Tokenizer
def create_and_train_tokenizer(texts):
    tokenizer=Tokenizer()
    tokenizer.fit_on_texts(texts)
    return tokenizer

from keras.preprocessing.sequence import pad_sequences
def encode_reviews(tokenizer, max_length, docs):
    encoded=tokenizer.texts_to_sequences(docs) 
    padded=pad_sequences(encoded, maxlen=max_length, padding="post")
    return padded

tokenizer=create_and_train_tokenizer(texts = X_train)
vocab_size=len(tokenizer.word_index) + 1
print("Vocabulary size:", vocab_size)

max_length=max([len(row.split()) for row in X_train])
print("Maximum length:",max_length)

X_train_encoded = encode_reviews(tokenizer, max_length, X_train)
X_test_encoded = encode_reviews(tokenizer, max_length, X_test)
print('x_train shape:', X_train_encoded.shape)
print('x_test shape:', X_test_encoded.shape)

Vocabulary size: 1182241
Maximum length: 16
x_train shape: (1502979, 16)
x_test shape: (59999, 16)


In [0]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb
from keras import layers, models
from keras.callbacks import EarlyStopping

In [11]:
from keras import layers, models

def create_embedding_model(vocab_size, max_length):
    model=models.Sequential()
    model.add(layers.Embedding(vocab_size, 100, input_length=max_length))

    model.add(layers.Conv1D(128, 5, activation="relu"))
    #model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D())

    model.add(layers.Conv1D(128, 5, activation="relu"))
    #model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D())

    model.add(layers.Flatten())
    model.add(layers.Dense(128,  activation="relu")) 
    dropout = Dropout(0.5)
    model.add(layers.Dense(15,  activation="softmax"))   
    return model

embedding_model = create_embedding_model(vocab_size=vocab_size, max_length=max_length)
embedding_model.summary()

from keras.optimizers import SGD
opt = SGD(lr=0.01, momentum=0.9)
embedding_model.compile(loss='categorical_crossentropy',
              optimizer= opt,
              metrics=['accuracy'])





Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 16, 100)           118224100 
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 12, 128)           64128     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 6, 128)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2, 128)            82048     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1, 128)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)          

In [12]:
print(len(y_train) + len(y_test))
print(len(X_train_encoded))

1562978
1502979


In [13]:
#earlyStopping = EarlyStopping(monitor="val_accuracy", patience=1)
modelHistory = embedding_model.fit(X_train_encoded, 
                                   y_train, 
                                   validation_data=(X_test_encoded, y_test),
                                   epochs= 40
                                   )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 1502979 samples, validate on 59999 samples
Epoch 1/40




/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:421: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 118224100 elements. This may consume a large amount of memory.
  num_elements)





1502979/1502979 [==============================] - 817s 544us/step - loss: 1.8209 - acc: 0.4290 - val_loss: 2.3965 - val_acc: 0.2990
Epoch 2/40
1502979/1502979 [==============================] - 799s 532us/step - loss: 1.5901 - acc: 0.5021 - val_loss: 2.1217 - val_acc: 0.3769
Epoch 3/40
1502979/1502979 [==============================] - 801s 533us/step - loss: 1.3272 - acc: 0.5838 - val_loss: 1.6552 - val_acc: 0.5206
Epoch 4/40
1123808/1502979 [=====================>........] - ETA: 3:21 - loss: 0.8005 - acc: 0.7535Buffered data was truncated after reaching the output size limit.

In [14]:
_, acc = embedding_model.evaluate(X_train_encoded, y_train, verbose=0)
print("Train accuracy:{:.2f}".format(acc*100))
_,acc= embedding_model.evaluate(X_test_encoded, y_test, verbose=0)
print("Test accuracy:{:.2f}".format(acc*100))

Train accuracy:97.55
Test accuracy:88.46
